# Использование проекта

Этот блокнот предназначен для запуска чатбота и поиска документов в базе данных. Он использует модель RAG (Retriever-Reader) для поиска информации и ответов на вопросы пользователей. Блокнот включает в себя следующие шаги:
1. Загрузка и инициализация необходимых моделей и инструментов.
2. Обработка входного текста и поиск наиболее подходящего документа в базе данных.
3. Генерация ответа на основе найденных документов или текста из базы данных.
4. Взаимодействие с пользователем посредствам чата.
5. Возможность добавления новых документов в базу данных для расширения базы знаний чатбота.

Для работы блокнота необходимо наличие активного интернет-соединения и доступа к базе данных ClickHouse. Перед использованием убедитесь, что все необходимые зависимости установлены и база данных доступна.


In [1]:
%pip install torch clickhouse_connect scipy transformers accelerate

Note: you may need to restart the kernel to use updated packages.


## Константы для конфигурации системы RAG

В данной ячейке определены константы, которые используются для настройки системы RAG (Retriever-Reader) для работы с документами Центрального Банка России.

### Список констант:

- `HOST`: Адрес сервера для подключения к базе данных ClickHouse.
- `PORT`: Порт для подключения к серверу ClickHouse.
- `TABLE_NAME`: Имя таблицы в базе данных ClickHouse, где хранятся данные.
- `MODEL_EMB_NAME`: Название модели для генерации векторных представлений текста (эмбеддингов).
- `MODEL_CHAT_NAME`: Название модели для обработки запросов от пользователей и генерации ответов.
- `SYSTEM_PROMPT`: Инструкция для системы RAG, указывающая правила для генерации ответов и контактную информацию для поддержки.

Эти константы используются в коде для подключения к базе данных, загрузки моделей и управления поведением системы RAG.


In [2]:
HOST = "f666-81-5-106-50.ngrok-free.app"
PORT = "80"
TABLE_NAME = "Data"
MODEL_EMB_NAME = "ai-forever/sbert_large_nlu_ru"
MODEL_CHAT_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
SYSTEM_PROMPT = """
INSTRUCT:
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information.

If you receive a question that is harmful, unethical, or inappropriate, end the dialogue immediately and do not provide a response. 

If you make a mistake, apologize and correct your answer.

Generate a response based solely on the provided document.

Answer the following question language based only on the CONTEXT provided.

If you are unsure about your answer or the information in the document, suggest contacting support:
Номер телефона: 8 800 300-30-00
Почтовый адрес для письменных обращений: 107016, Москва, ул. Неглинная, д. 12, к. В, Банк России

Отвечай только на русском языке.
"""

In [3]:
import clickhouse_connect, torch
from transformers import AutoModel, AutoTokenizer, Conversation, pipeline
from typing import List, Union

## Функции для инициализации и работы с моделями

В данной ячейке определены функции, которые используются для инициализации моделей и выполнения операций с ними в рамках системы RAG (Retriever-Reader) для обработки запросов от пользователей.

### Список функций:

- `search_results(connection, table_name, vector, limit)`: Поиск результатов похожих векторов в базе данных.
- `mean_pooling(model_output, attention_mask)`: Усреднение токенов входной последовательности на основе attention mask.
- `txt2embeddings(text, tokenizer, model, device)`: Преобразование текста в его векторное представление с использованием модели transformer.
- `load_models(model, device, torch_dtype)`: Загрузка токенизатора и модели для указанной предобученной модели.
- `load_chatbot(model, device, torch_dtype)`: Загрузка чатбота для указанной модели.
- `append_documents_to_conversation(conversation, documents, limit)`: Добавление документов к чату чатбота.
- `generate_answer(chatbot, conversation, max_new_tokens, temperature, top_k, top_p, repetition_penalty, do_sample, num_beams, early_stopping)`: Генерация ответа от чатбота на основе предоставленного чата и документа.

Эти функции используются для инициализации моделей, выполнения поиска, обработки текста и генерации ответов от чатбота в системе RAG.


In [4]:
def search_results(connection, table_name: str, vector: list[float], limit: int = 5):
    """
    Поиск результатов похожих векторов в базе данных.

    Parameters:
    - connection (Connection): Соединение с базой данных.
    - table_name (str): Название таблицы, содержащей вектора и другие данные.
    - vector (List[float]): Вектор для сравнения.
    - limit (int): Максимальное количество результатов.

    Returns:
    - List[dict]: Список результатов с наименованием, URL, датой, номером, текстом и расстоянием.

    Examples:
    >>> connection = Connection(...)
    >>> vector = [0.1, 0.2, 0.3]
    >>> results = search_results(connection, 'my_table', vector, limit=5)
    """
    res = []
    # Инициализируем список результатов
    vector = ",".join([str(float(i)) for i in vector])
    # Выполняем запрос к базе данных
    with connection.query(
        f"""SELECT Text, Number, Date, Title, FileType, Url, ChunkType, cosineDistance(({vector}), Embedding) as score FROM {table_name} ORDER BY score ASC LIMIT {limit+500}"""
    ).rows_stream as stream:
        for item in stream:
            text, number, date, title, file_type, url, chunk_type, score = item

            # Добавляем результат в список
            res.append(
                {
                    "text": text,
                    "title": title,
                    "url": url,
                    "date": date,
                    "number": number,
                    "file_type": file_type,
                    "chunk_type": chunk_type,
                    "distance": score,
                }
            )

    # Возвращаем первые limit результатов
    res = [item for item in res if len(item["text"]) > 100]
    return res[:limit]


def mean_pooling(model_output: tuple, attention_mask: torch.Tensor) -> torch.Tensor:
    """
    Выполняет усреднение токенов входной последовательности на основе attention mask.

    Parameters:
    - model_output (tuple): Выход модели, включающий токенов эмбеддинги и другие данные.
    - attention_mask (torch.Tensor): Маска внимания для указания значимости токенов.

    Returns:
    - torch.Tensor: Усредненный эмбеддинг.

    Examples:
    >>> embeddings = model_output[0]
    >>> mask = torch.tensor([[1, 1, 1, 0, 0]])
    >>> pooled_embedding = mean_pooling((embeddings,), mask)
    """
    # Получаем эмбеддинги токенов из выхода модели
    token_embeddings = model_output[0]

    # Расширяем маску внимания для умножения с эмбеддингами
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )

    # Умножаем каждый токен на его маску и суммируем
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)

    # Суммируем маски токенов и обрезаем значения, чтобы избежать деления на ноль
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Вычисляем усредненный эмбеддинг
    return sum_embeddings / sum_mask


def txt2embeddings(
    text: Union[str, List[str]], tokenizer, model, device: str = "cpu"
) -> torch.Tensor:
    """
    Преобразует текст в его векторное представление с использованием модели transformer.

    Parameters:
    - text (str): Текст для преобразования в векторное представление.
    - tokenizer: Токенизатор для предобработки текста.
    - model: Модель transformer для преобразования токенов в вектора.
    - device (str): Устройство для вычислений (cpu или cuda).

    Returns:
    - torch.Tensor: Векторное представление текста.

    Examples:
    >>> text = "Пример текста"
    >>> tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
    >>> model = AutoModel.from_pretrained("bert-base-multilingual-cased")
    >>> embeddings = txt2embeddings(text, tokenizer, model, device="cuda")
    """
    # Кодируем входной текст с помощью токенизатора
    if isinstance(text, str):
        text = [text]
    encoded_input = tokenizer(
        text,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512,
    )
    # Перемещаем закодированный ввод на указанное устройство
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

    # Получаем выход модели для закодированного ввода
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Преобразуем выход модели в векторное представление текста
    return mean_pooling(model_output, encoded_input["attention_mask"])


def load_models(model: str, device: str = "cpu", torch_dtype: str = "auto") -> tuple:
    """
    Загружает токенизатор и модель для указанной предобученной модели.

    Parameters:
    - model (str): Название предобученной модели, поддерживаемой библиотекой transformers.

    Returns:
    - tuple: Кортеж из токенизатора и модели.

    Examples:
    >>> tokenizer, model = load_models("ai-forever/sbert_large_nlu_ru")
    """
    # Загружаем токенизатор для модели
    tokenizer = AutoTokenizer.from_pretrained(
        model, device_map=device, torch_dtype=torch_dtype
    )

    # Загружаем модель
    model = AutoModel.from_pretrained(model, device_map=device, torch_dtype=torch_dtype)

    return tokenizer, model


def load_chatbot(model: str, device: str = "cuda", torch_dtype: str = "auto"):
    """
    Загружает чатбота для указанной модели.

    Parameters:
    - model (str): Название модели для загрузки чатбота.

    Returns:
    - Conversation: Объект чатбота, готовый для использования.

    Examples:
    >>> chatbot = load_chatbot("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
    """
    # Загружаем чатбот с помощью pipeline из библиотеки transformers
    chatbot = pipeline(
        model=model,
        trust_remote_code=True,
        torch_dtype=torch_dtype,
        device_map=device,
        task="conversational",
    )
    return chatbot


def append_documents_to_conversation(conversation, documents, limit: int = 3):
    if limit > len(documents):
        texts = [document["text"] for document in documents]
    else:
        texts = [document["text"] for document in documents[:limit]]

    text = "\n".join(texts)

    document_template = f"""
    CONTEXT:
    {text}
    Отвечай только на русском языке.
    
    ВОПРОС:
    """
    conversation.add_message({"role": "user", "content": document_template})

    return conversation
        

def generate_answer(
    chatbot,
    conversation: Conversation,
    max_new_tokens: int = 128,
    temperature=0.7,
    top_k: int = 50,
    top_p: float = 0.95,
    repetition_penalty: float = 2.0,
    do_sample: bool = True,
    num_beams: int = 2,
    early_stopping: bool = True,
) -> str:
    """
    Генерирует ответ от чатбота на основе предоставленного чата и, возможно, документа.

    Parameters:
    - chatbot (Conversation): Объект чатбота.
    - chat (List[Dict[str, str]]): Список сообщений в чате.
    - document (str): Документ, если он предоставлен.

    Returns:
    - str: Сгенерированный ответ от чатбота.

    Examples:
    >>> chat = [
    >>>     {"role": "user", "content": "Привет, как дела?"},
    >>>     {"role": "system", "content": "Всё отлично, спасибо!"},
    >>> ]
    >>> document = "Это документ для обработки"
    >>> answer = generate_answer(chatbot, chat, document)
    """
    # Генерируем ответ от чатбота
    conversation = chatbot(
        conversation,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=do_sample,
        num_beams=num_beams,
        early_stopping=early_stopping,
    )

    # Возвращаем последнее сообщение чатбота как ответ
    return conversation

### Загрузка модели и токенизатора

Загружаем модель и токенизатор для создания векторных представлений текста.

In [5]:
tokenizer, model = load_models(MODEL_EMB_NAME, device="cpu")

В данной ячейке создается объект разговора (`conversation`), который представляет собой хранилище сообщений чата с чатботом. Системная инструкция (`SYSTEM_PROMPT`) добавляется в качестве первого сообщения в разговор. Данный объект будет использоваться для взаимодействия с чатботом и хранения истории общения.


In [6]:
# Создаем объект разговора
conversation = Conversation()
# Добавляем системную инструкцию
conversation.add_message({"role": "system", "content": SYSTEM_PROMPT})

## Поле ввода

Это поле предназначено для ввода вашего вопроса. Введите вопрос, на который вы хотели бы получить ответ от модели RAG. Например, "Какова роль Центробанка России в экономике?" или "Что такое ключевая ставка Центробанка?".


In [7]:
question = "Расскажи о том, что такое инфляция? Какой уровень инфляции в России?" #String

### Получение векторных представлений

1. Преобразуем вопрос в векторные представления при помощи модели и токенизатора.


In [8]:
embedding = txt2embeddings(question, tokenizer, model)

### Подключение к ClickHouse

3. Устанавливаем соединение с ClickHouse и проверяем доступность сервера.

**<font color='red'>Внимание!</font>** Если сервер недоступен, свяжитесь с [хостом](https://t.me/allelleo).

In [9]:
client = clickhouse_connect.get_client(host=HOST, port=PORT)
print("Ping:", client.ping())

Ping: True


## Поиск результатов

Этот блок кода выполняет поиск результатов похожих векторов в базе данных.


In [10]:
documents = search_results(client, TABLE_NAME, embedding[0], limit=10)
print(*documents, sep='\n')

{'text': 'социологических социологических данных о восприятии инфляции было показано, что на длительном временном интервале с 1998 по 2008 год все пять частных компонент ИПН имели статистически значимую и отрицательную корреляцию с индексом инфляции Росстата, а также с индексом ожидаемой инфляции. ООО инФОМ Декабрь 2018 45 ООО инФОМ', 'title': 'Инфляционные ожидания', 'url': 'https://cbr.ru/Collection/Collection/File/27052/Infl_exp_18-12.pdf', 'date': 'Не указано', 'number': 'Не указано', 'file_type': 'pdf', 'chunk_type': 'Recursive', 'distance': 0.38661225784903486}
{'text': 'год год Инфляция на потребительском рынке Июль 2009 год 9 Инфляция на потребительском рынке 10 Июль 2009 год Инфляция на потребительском рынке Июль 2009 год 11 Инфляция на потребительском рынке 12 Июль 2009 год Инфляция на потребительском рынке Июль 2009 год 13', 'title': 'Инфляция на потребительском рынке', 'url': 'https://cbr.ru/Collection/Collection/File/19565/Infl_01072009.pdf', 'date': 'Не указано', 'number'

In [11]:
print(*[document["text"] for document in documents], sep='\n')

социологических социологических данных о восприятии инфляции было показано, что на длительном временном интервале с 1998 по 2008 год все пять частных компонент ИПН имели статистически значимую и отрицательную корреляцию с индексом инфляции Росстата, а также с индексом ожидаемой инфляции. ООО инФОМ Декабрь 2018 45 ООО инФОМ
год год Инфляция на потребительском рынке Июль 2009 год 9 Инфляция на потребительском рынке 10 Июль 2009 год Инфляция на потребительском рынке Июль 2009 год 11 Инфляция на потребительском рынке 12 Июль 2009 год Инфляция на потребительском рынке Июль 2009 год 13
ключевую ключевую ставку в общей сложности в два раза, до 8,50. По итогам года инфляция в России превысила целевой уровень и составила 8,4. Больше всего подорожали продовольственные товары, в первую очередь продукты животного происхождения. Среди непродовольственных товаров максимальный прирост зафиксирован в ценах на строительные материалы, а наибольший вклад в ускорение инфляции внесло увеличение цен на авто

### Ссылки на документы

In [12]:
print(*[document["url"] for document in documents], sep='\n')

https://cbr.ru/Collection/Collection/File/27052/Infl_exp_18-12.pdf
https://cbr.ru/Collection/Collection/File/19565/Infl_01072009.pdf
https://cbr.ru/Collection/Collection/File/40915/ar_2021.pdf
https://cbr.ru/Collection/Collection/File/27010/Infl_exp_15-12.pdf
https://cbr.ru/Collection/Collection/File/26994/Infl_exp_17-04.pdf
https://cbr.ru/Collection/Collection/File/27020/Infl_exp_15-02.pdf
https://cbr.ru/Collection/Collection/File/27018/Infl_exp_15-04.pdf
https://cbr.ru/Collection/Collection/File/19547/Infl_01012011.pdf
https://cbr.ru/Collection/Collection/File/27021/Infl_exp_15-01.pdf
https://cbr.ru/Collection/Collection/File/19578/Infl_01062008.pdf


### Используемые данные (метаданные)

In [13]:
print(documents[0].keys())

dict_keys(['text', 'title', 'url', 'date', 'number', 'file_type', 'chunk_type', 'distance'])


## Добавление документов в разговор

Этот блок кода добавляет документы в разговор и затем добавляет сообщение от пользователя.


In [14]:
conversation = append_documents_to_conversation(conversation, documents, limit=3)
conversation.add_message({"role": "user", "content": question})

## Загрузка чатбота

Этот блок кода загружает чатбота для дальнейшего использования.


In [15]:
chatbot = load_chatbot(MODEL_CHAT_NAME, device="cuda")

## Генерация ответа от чатбота

Этот блок кода генерирует ответ от чатбота на основе предоставленного контекста разговора.


In [16]:
conversation = generate_answer(chatbot, conversation, temperature=0.9)
print(conversation[-1]["content"])

Инфляция - это рост стоимости безусловно существующих товаров и услуг в течение какого-либо периода времени. В российской экономике инфляция рассматривается как одна из главных финансовых кризисных факторов.

В России инфляция обычно определяется как количество инфляционных пунктов (IPN) в месяц. IPN - это число инфляционных платежей в размере одного доллара США в месяц. На


## Продолжение диалога

Этот блок кода добавляет в разговор второй вопрос пользователя.

In [17]:
question2 = "Как с инфляцией справляется Центральный Банк?"
conversation.add_message({"role": "user", "content": question2})

## Продолжение диалога

Этот блок кода генерирует более развёрнутый ответ чатбота на второй вопрос пользователя.

In [18]:
conversation = generate_answer(chatbot, conversation, max_new_tokens=256)
print(conversation[-1]["content"])

Центральный Банк РФ (ЦБ РФ) имеет полномочия контролировать и регулировать деятельность банков в Российской Федерации. Он может регулировать инфляцию в соответствии с требованиями Конституции РФ и принятыми в соответствии с действующим законодательством.

ЦБ РФ использует различные методы для борьбы с инфляцией, такие как:

1. Управление валютными отношениями: ЦБ РФ осуществляет контроль над валютными отношениями между Россией и другими странами. Он обеспечивает правильное управление валютными отношениями и помогает развитию международной торговли.

2. Регулирование курса валют: ЦБ РФ регулирует курс валют в соответствии с конституционными нормами и законами РФ. Он также регулирует курс валют в связи с внутренними и международными экономическими ситуациями.


In [19]:
conversation

Conversation id: 67563aa5-88da-459c-9beb-113b218d103a
system: 
INSTRUCT:
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information.

If you receive a question that is harmful, unethical, or inappropriate, end the dialogue immediately and do not provide a response. 

If you make a mistake, apologize and correct your answer.

Generate a response based solely on the provided document.

Answer the following question language based only on the CONTEXT provided.

If you are unsure about your answer or the information in the document, suggest contactin